<a href="https://colab.research.google.com/github/EjbejaranosAI/EmotionUnify/blob/main/vision_Meld_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download andinstall libraries and dataset

In [1]:
!pip install -q lazypredict
!pip install -q opencv-python
!pip install -q torch torchvision
!pip install -q transformers
!pip install -q timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 31.6 MB/s eta 0:00:00


### Module to set the dataset

In [2]:
"""
This module is to download multimodal datasets and store them in the dataset directory
"""
import os
import os.path
from os import path
import logging
import shutil
import subprocess
import tarfile


def download_meld_dataset():
    """
    Download MELD dataset and store it into the dataset directory

    """
    if path.exists("/content/dataset")== False:
      os.mkdir("/content/datasets")

    os.chdir('/content/datasets')
    try:
        # Add logging configuration
        logging.basicConfig(filename='download.log', level=logging.INFO)
        logging.info('Downloading MELD dataset')

        # Use wget to download the dataset
        try:
            subprocess.run(['wget', 'https://huggingface.co/datasets/declare-lab/MELD/resolve/main/MELD.Raw.tar.gz'])
        except:
            subprocess.run(['wget','http://web.eecs.umich.edu/~mihalcea/downloads/MELD.Raw.tar.gz'])

        # Extract the downloaded dataset
        subprocess.run(['tar', '-xvf', 'MELD.Raw.tar.gz'])

        # Remove the downloaded tar.gz file
        os.remove('MELD.Raw.tar.gz')

        # Rename the extracted directory to "MELD"
        os.rename('MELD.Raw', 'MELD')

        # Extract the inner tar files
        files = os.listdir("./MELD")
        files_to_uncompress = []
        for file in files:
            if file.endswith('.tar.gz'):
                print("tar.gz file found")
                files_to_uncompress.append(file)
        if len(files_to_uncompress) >0:
            for file in files_to_uncompress:
                # I want to extract the files in the same directory that was found it and later deleter the tar.gz files
                tar = tarfile.open("./MELD/"+file)
                tar.extractall("./MELD/")
                tar.close()
                os.remove("./MELD/"+file)

        else:
            print("No tar.gz files found")
    except:
        logging.error('Error in downloading MELD dataset')
        print('Error in downloading MELD dataset')


if __name__ == "__main__":
    download_meld_dataset()

tar.gz file found
tar.gz file found
tar.gz file found


# Reading dataset

In [ ]:
import os
import numpy as np
import pandas as pd

path_dataset = "/content/datasets/MELD"

# In[2]:
def load_data(path):
    csv_file_data = []
    files_to_uncompress = []
    files = os.listdir(path)
    print(files)
    # get the csv files from the files and read it
    for file in files:
        if file.endswith(".csv"):
            print(file)
            csv_file_data.append({file:pd.read_csv(path + '/' + file)})


    print(len(csv_file_data))



    return files, csv_file_data

files , csv_file_data = load_data(path_dataset)

['._train_splits', '._output_repeated_splits_test', 'output_repeated_splits_test', 'dev_sent_emo.csv', 'train_splits', 'test_sent_emo.csv', 'README.txt', 'dev_splits_complete', 'train_sent_emo.csv']
dev_sent_emo.csv
test_sent_emo.csv
train_sent_emo.csv
3


In [ ]:
train_df = csv_file_data[2]['train_sent_emo.csv']
dev_df = csv_file_data[0]['dev_sent_emo.csv']
test_df = csv_file_data[1]['test_sent_emo.csv']

In [ ]:
df_train = train_df
df_dev = dev_df
df_test = test_df
print(len(df_train))
df_train.head()

9989


,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,also I was the point person on my companys tr...,Chandler,neutral,neutral,0,0,8,21,"00:16:16,059","00:16:21,731"
1,2,You mustve had your hands full.,The Interviewer,neutral,neutral,0,1,8,21,"00:16:21,940","00:16:23,442"
2,3,That I did. That I did.,Chandler,neutral,neutral,0,2,8,21,"00:16:23,442","00:16:26,389"
3,4,So lets talk a little bit about your duties.,The Interviewer,neutral,neutral,0,3,8,21,"00:16:26,820","00:16:29,572"
4,5,My duties? All right.,Chandler,surprise,positive,0,4,8,21,"00:16:34,452","00:16:40,917"


In [ ]:
# Define mapping dictionaries
emotion_mapping = {
    'neutral': [1, 0, 0, 0, 0, 0, 0],
    'surprise': [0, 1, 0, 0, 0, 0, 0],
    'fear': [0, 0, 1, 0, 0, 0, 0],
    'sadness': [0, 0, 0, 1, 0, 0, 0],
    'joy': [0, 0, 0, 0, 1, 0, 0],
    'disgust': [0, 0, 0, 0, 0, 1, 0],
    'anger': [0, 0, 0, 0, 0, 0, 1]
}

sentiment_mapping = {
    'neutral': 0,
    'positive': 1,
    'negative': 2
}

In [ ]:
# Process the training, development, and test datasets
df_train['video_id'] = df_train.apply(lambda row: f'dia{row["Dialogue_ID"]}_utt{row["Utterance_ID"]}.mp4', axis=1)
df_dev['video_id'] = df_dev.apply(lambda row: f'dia{row["Dialogue_ID"]}_utt{row["Utterance_ID"]}.mp4', axis=1)
df_test['video_id'] = df_test.apply(lambda row: f'dia{row["Dialogue_ID"]}_utt{row["Utterance_ID"]}.mp4', axis=1)

df_train['Emotion_encoded'] = df_train['Emotion'].map(emotion_mapping)
df_train['Sentiment_encoded'] = df_train['Sentiment'].map(sentiment_mapping)
df_processed_train = df_train[['video_id', 'Utterance', 'Emotion_encoded', 'Sentiment_encoded']]

df_dev['Emotion_encoded'] = df_dev['Emotion'].map(emotion_mapping)
df_dev['Sentiment_encoded'] = df_dev['Sentiment'].map(sentiment_mapping)
df_processed_dev = df_dev[['video_id', 'Utterance', 'Emotion_encoded', 'Sentiment_encoded']]

df_test['Emotion_encoded'] = df_test['Emotion'].map(emotion_mapping)
df_test['Sentiment_encoded'] = df_test['Sentiment'].map(sentiment_mapping)
df_processed_test = df_test[['video_id', 'Utterance', 'Emotion_encoded', 'Sentiment_encoded']]

In [ ]:
df_processed_train

,video_id,Utterance,Emotion_encoded,Sentiment_encoded
0,dia0_utt0.mp4,also I was the point person on my companys tr...,"[1, 0, 0, 0, 0, 0, 0]",0
1,dia0_utt1.mp4,You mustve had your hands full.,"[1, 0, 0, 0, 0, 0, 0]",0
2,dia0_utt2.mp4,That I did. That I did.,"[1, 0, 0, 0, 0, 0, 0]",0
3,dia0_utt3.mp4,So lets talk a little bit about your duties.,"[1, 0, 0, 0, 0, 0, 0]",0
4,dia0_utt4.mp4,My duties? All right.,"[0, 1, 0, 0, 0, 0, 0]",1
...,...,...,...,...
9984,dia1038_utt13.mp4,You or me?,"[1, 0, 0, 0, 0, 0, 0]",0
9985,dia1038_utt14.mp4,"I got it. Uh, Joey, women don't have Adam's ap...","[1, 0, 0, 0, 0, 0, 0]",0
9986,dia1038_utt15.mp4,"You guys are messing with me, right?","[0, 1, 0, 0, 0, 0, 0]",1
9987,dia1038_utt16.mp4,Yeah.,"[1, 0, 0, 0, 0, 0, 0]",0


# Feature extraction in text

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm  # Import tqdm for progress bars

# Define a function to extract BERT features
def extract_bert_features(df):
    # Tokenize utterances and obtain BERT features with padding
    max_length = 128  # Define your desired maximum sequence length

    # Tokenize utterances and obtain BERT features
    utterances = df['Utterance'].tolist()
    input_ids = []
    attention_masks = []

    # Use tqdm to add progress bars
    for utterance in tqdm(utterances, desc="Tokenizing Utterances"):
        tokens = tokenizer.encode(utterance, add_special_tokens=True, max_length=max_length, truncation=True, padding='max_length')
        input_ids.append(tokens)
        attention_masks.append([1] * len(tokens))

    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    with torch.no_grad():
        bert_outputs = model(input_ids, attention_mask=attention_masks)

    # Use BERT features to train a logistic regression classifier
    X = bert_outputs[0][:, 0, :].numpy()  # Using the [CLS] token's representation
    y = df['Sentiment_encoded'].tolist()

    return X, y

In [ ]:
# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


# Extract BERT features for the training, development, and test datasets
X_train, y_train = extract_bert_features(df_processed_train)
np.save('train_text_features.npy', X_train)

X_dev, y_dev = extract_bert_features(df_processed_dev)
np.save('dev_text_features.npy', X_dev)

X_test, y_test = extract_bert_features(df_processed_test)
np.save('test_text_features.npy', X_test)

Tokenizing Utterances: 100%|██████████| 2610/2610 [00:00<00:00, 3851.59it/s]


### Text classifier

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Convert one-hot encoded labels to single-class labels
#y_train_single = np.argmax(y_train, axis=1)
#y_test_single = np.argmax(y_test, axis=1)


classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)

# Evaluate the classifier
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.6754789272030651


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

 97%|█████████▋| 28/29 [12:34<00:35, 35.52s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.040925 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 195839
[LightGBM] [Info] Number of data points in the train set: 9989, number of used features: 768
[LightGBM] [Info] Start training from score -0.751797
[LightGBM] [Info] Start training from score -1.453901
[LightGBM] [Info] Start training from score -1.221376


100%|██████████| 29/29 [12:45<00:00, 26.41s/it]

                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
LogisticRegression                 0.67               0.63    None      0.67   
LinearDiscriminantAnalysis         0.66               0.62    None      0.65   
RidgeClassifierCV                  0.67               0.62    None      0.66   
RidgeClassifier                    0.66               0.61    None      0.65   
LinearSVC                          0.64               0.60    None      0.63   
CalibratedClassifierCV             0.65               0.59    None      0.64   
SGDClassifier                      0.61               0.58    None      0.60   
XGBClassifier                      0.64               0.57    None      0.62   
SVC                                0.65               0.57    None      0.62   
NuSVC                              0.65               0.57    None      0.63   
QuadraticDiscriminantAnalysis      0.65 

In [ ]:
models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LogisticRegression,0.67,0.63,None,0.67,1.84
LinearDiscriminantAnalysis,0.66,0.62,None,0.65,2.19
RidgeClassifierCV,0.67,0.62,None,0.66,1.57
RidgeClassifier,0.66,0.61,None,0.65,0.70
LinearSVC,0.64,0.60,None,0.63,65.27
CalibratedClassifierCV,0.65,0.59,None,0.64,261.63
SGDClassifier,0.61,0.58,None,0.60,7.53
XGBClassifier,0.64,0.57,None,0.62,104.50
SVC,0.65,0.57,None,0.62,51.76


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(models, *['Accuracy'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(models, *['Balanced Accuracy'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(models, *['F1 Score'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(models, *['Time Taken'], **{})
chart

...

import numpy as np
from google.colab import autoviz

def scatter_plots(df, colname_pairs, figscale=1, alpha=.8):
  from matplotlib import pyplot as plt
  plt.figure(figsize=(len(colname_pairs) * 6 * figscale, 6 * figscale))
  for plot_i, (x_colname, y_colname) in enumerate(colname_pairs, start=1):
    ax = plt.subplot(1, len(colname_pairs), plot_i)
    df.plot(kind='scatter', x=x_colname, y=y_colname, s=(32 * figscale), alpha=alpha, ax=ax)
    ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = scatter_plots(models, *[[['Accuracy', 'Balanced Accuracy'], ['Balanced Accuracy', 'F1 Score'], ['F1 Score', 'Time Taken']]], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(models, *['Time Taken', 'Accuracy', None], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(models, *['Time Taken', 'Balanced Accuracy', None], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(models, *['Time Taken', 'F1 Score', None], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(models, *['Time Taken', 'count()', None], **{})
chart

In [ ]:
import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(models, *['Accuracy'], **{})
chart

# Feature extraction in audio

In [ ]:
import torch
import torch.nn as nn
from transformers import Wav2Vec2Processor
from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2Model,
    Wav2Vec2PreTrainedModel,
)
import torchaudio


class RegressionHead(nn.Module):
    r"""Classification head."""

    def __init__(self, config):

        super().__init__()

        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):

        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)

        return x


class EmotionModel(Wav2Vec2PreTrainedModel):
    r"""Speech emotion classifier."""

    def __init__(self, config):

        super().__init__(config)

        self.config = config
        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = RegressionHead(config)
        self.init_weights()

    def forward(
            self,
            input_values,
    ):

        outputs = self.wav2vec2(input_values)
        hidden_states = outputs[0]
        hidden_states = torch.mean(hidden_states, dim=1)
        logits = self.classifier(hidden_states)

        return hidden_states, logits


def process_func(
    wavs,
    sampling_rate: int
    # embeddings: bool = False,
):
    r"""Predict emotions or extract embeddings from raw audio signal."""

    # run through processor to normalize signal
    # always returns a batch, so we just get the first entry
    # then we put it on the device
    # wavs = pad_sequence(wavs, batch_first=True)
    # load model from hub
    device = 'cpu'
    model_name = 'audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim'
    processor = Wav2Vec2Processor.from_pretrained(model_name)
    model = EmotionModel.from_pretrained(model_name)

    y = processor([wav.cpu().numpy() for wav in wavs],
                   sampling_rate=sampling_rate,
                   return_tensors="pt",
                   padding="longest"
        )
    y = y['input_values']
    y = y.to(device)


    y = model(y)

    return {
        'hidden_states': y[0],
        'logits': y[1],
    }




### Extracting features for one audio

In [ ]:
!pip install -q pydub

### Extracting features in all the dataset

 version optimized

In [ ]:
from multiprocessing import Pool
import torch
from tqdm import tqdm
from pydub import AudioSegment
import moviepy.editor as mpe
import numpy as np
import os

# Your existing classes and functions
# ...

def extract_audio_features_from_video(video_path):
    audio_features_placeholder = torch.zeros(1, 768)
    try:
        video_clip = mpe.VideoFileClip(video_path)
        audio_clip = video_clip.audio
        audio_file = "/content/audio.wav"
        audio_clip.write_audiofile(audio_file, codec='pcm_s16le')
        sampling_rate = 16000
        audio = AudioSegment.from_file(audio_file)
        signal = np.array(audio.get_array_of_samples())
        signal = signal.reshape(-1, 2).T
        signal = torch.from_numpy(signal).float()
        with torch.no_grad():
            audio_features = process_func(signal, sampling_rate)
        return audio_features['hidden_states']
    except Exception as e:
        print(f"Error: {e}")
        return audio_features_placeholder

def process_videos(video_ids):
    audio_features = []
    for video_id in tqdm(video_ids):
        video_path = os.path.join(video_folder, video_id)
        audio_features.append(extract_audio_features_from_video(video_path))
    return np.concatenate(audio_features, axis=0)


video_folder = "/content/datasets/MELD/output_repeated_splits_test"
video_id_files = df_processed_test['video_id'].tolist()

# Using 4 processes for parallel execution
with Pool(2) as p:
    all_audio_features = p.map(process_videos, np.array_split(video_id_files, 2))

#all_audio_features = process_videos(video_id_files)
# Concatenate the extracted audio features from all processes
audio_features = np.concatenate(all_audio_features, axis=0)

# Save the extracted audio features to a file.
np.save('test_audio_features.npy', audio_features)


MoviePy - Writing audio in /content/audio.wav


MoviePy - Writing audio in /content/audio.wav

chunk:   0%|          | 0/138 [00:00<?, ?it/s, now=None]


MoviePy - Done.


  0%|          | 0/1305 [00:00<?, ?it/s]


...

MoviePy - Writing audio in /content/audio.wav


chunk:  97%|█████████▋| 5022/5184 [00:02<00:00, 1753.75it/s, now=None]


MoviePy - Done.


 57%|█████▋    | 747/1305 [1:27:40<51:36,  5.55s/it]Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Process ForkPoolWorker-3:
Process ForkPoolWorker-4:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File "/usr/lib/python3.10/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in ru

KeyboardInterrupt: ignored

# Video extraction

In [ ]:
import cv2
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision.models import resnet18, vgg16
import timm
from tqdm import tqdm



# Check for available GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class EarlyFusionLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(EarlyFusionLayer, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim).to(device)

    def forward(self, x):
        return self.fc(x)

def initialize_models():
    models = {
        'resnet': resnet18(pretrained=True).to(device),
        'vgg': vgg16(pretrained=True).to(device),
        'vit': timm.create_model("vit_base_patch16_224", pretrained=True).to(device)
    }
    for key in models:
        if key != 'vit':
            models[key] = nn.Sequential(*(list(models[key].children())[:-1])).to(device)
    return models

def extract_frames(video_path):
    frames = []
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return frames

def extract_features_from_frame(model, frame):
    frame = cv2.resize(frame, (224, 224))
    tensor = torch.tensor(frame).float().unsqueeze(0).permute(0, 3, 1, 2).to(device) / 255.0
    with torch.no_grad():
        features = model(tensor)
    return np.squeeze(features.cpu().numpy())

def process_video(video_path, models, fusion_layer):
    frames = extract_frames(video_path)
    combined_features = []

    for model_name, model in models.items():
        frame_features = [extract_features_from_frame(model, frame) for frame in frames]
        frame_features = np.stack(frame_features).reshape(len(frames), -1)
        combined_features.append(frame_features)

    combined_features = np.concatenate(combined_features, axis=1)
    combined_features_tensor = torch.tensor(combined_features).float().to(device)

    with torch.no_grad():
        fused_features = fusion_layer(combined_features_tensor)

    return fused_features.cpu().numpy()

def get_features_from_folder_videos(video_folder_path, filename_npy, df):
    models = initialize_models()
    video_folder = video_folder_path

    # Filter the video_files based on the DataFrame
    available_video_files = set(os.listdir(video_folder))
    df_video_files = set(df['video_id'])
    video_files_to_process = list(available_video_files.intersection(df_video_files))

    early_fusion_layer = EarlyFusionLayer(input_dim=26600, output_dim=1024).to(device)

    video_feature_dict = {}
    for video_file in tqdm(video_files_to_process):  # Change here to use the filtered list
        video_path = os.path.join(video_folder, video_file)
        features = process_video(video_path, models, early_fusion_layer)
        video_feature_dict[video_file] = features

    np.save(f'{filename_npy}_video_features.npy', video_feature_dict)

if __name__ == "__main__":
    get_features_from_folder_videos("/content/datasets/MELD/train_splits", "train", df_processed_train)
    get_features_from_folder_videos("/content/datasets/MELD/dev_splits_complete", "dev", df_processed_dev)
    get_features_from_folder_videos("/content/datasets/MELD/output_repeated_splits_test", "test", df_processed_test)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
 

# Vision feature extraction 2

In [3]:
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision.models import resnet18, vgg16  # Asegúrate de importar los modelos correctamente
import cv2
import timm
from tqdm import tqdm
import pandas as pd

class VisionFeatureExtractor:
    def __init__(self, classification_type="Sentiment"):
        self.input_dim = input_dim = 26600
        self.output_dim = output_dim = 1024
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.models = self.initialize_models()
        self.early_fusion_layer = EarlyFusionLayer(input_dim=self.input_dim, output_dim=self.output_dim).to(self.device)
        self.classification_type = classification_type

    def initialize_models(self):
        models = {
            'resnet': resnet18(pretrained=True).to(self.device),
            'vgg': vgg16(pretrained=True).to(self.device),
            'vit': timm.create_model("vit_base_patch16_224", pretrained=True).to(self.device)
        }
        for key in models:
            if key != 'vit':
                models[key] = nn.Sequential(*(list(models[key].children())[:-1]))
        return models

    def mapping_emotion(self, df):
        emotion_mapping = {
            'neutral': [1, 0, 0, 0, 0, 0, 0],
            'surprise': [0, 1, 0, 0, 0, 0, 0],
            'fear': [0, 0, 1, 0, 0, 0, 0],
            'sadness': [0, 0, 0, 1, 0, 0, 0],
            'joy': [0, 0, 0, 0, 1, 0, 0],
            'disgust': [0, 0, 0, 0, 0, 1, 0],
            'anger': [0, 0, 0, 0, 0, 0, 1]
        }
        df['Emotion_encoded'] = df['Emotion'].map(emotion_mapping)
        return df

    def mapping_sentiment(self, df):
        sentiment_mapping = {
            'neutral': 0,
            'positive': 1,
            'negative': 2
        }
        df['Sentiment_encoded'] = df['Sentiment'].map(sentiment_mapping)
        return df

    def early_fusion(self, features_list):
        combined_features = np.concatenate(features_list, axis=1)
        combined_features_tensor = torch.tensor(combined_features).float().to(self.device)
        with torch.no_grad():
            fused_features = self.early_fusion_layer(combined_features_tensor)
        return fused_features.cpu().numpy()

    def extract_features_from_video(self, video_path):
        frames = self.extract_frames(video_path)
        features_list = []
        for model_name, model in self.models.items():
            frame_features = [self.extract_features_from_frame(model, frame) for frame in frames]
            features_list.append(np.stack(frame_features).reshape(len(frames), -1))
        return self.early_fusion(features_list)

    def extract_frames(self, video_path):
        frames = []
        cap = cv2.VideoCapture(video_path)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)
        cap.release()
        return frames

    def extract_features_from_frame(self, model, frame):
        frame = cv2.resize(frame, (224, 224))
        tensor = torch.tensor(frame).float().unsqueeze(0).permute(0, 3, 1, 2) / 255.0
        tensor = tensor.to(self.device)
        with torch.no_grad():
            features = model(tensor)
        return np.squeeze(features.cpu().numpy())

    def extract_features_from_folder(self, video_folder_path, df):
        print("Entering into the extractor")
        video_feature_dict = {}
        video_files_to_process = [file for file in os.listdir(video_folder_path) if file in set(df['video_id'])]
        print(f"======{video_files_to_process}")

        if self.classification_type == "Emotion":
            df = self.mapping_emotion(df)
            label_column = 'Emotion_encoded'
        elif self.classification_type == "Sentiment":
            df = self.mapping_sentiment(df)
            label_column = 'Sentiment_encoded'

        for video_file in tqdm(video_files_to_process):
            video_path = os.path.join(video_folder_path, video_file)
            features = self.extract_features_from_video(video_path)
            label = df[df['video_id'] == video_file][label_column].values[0]
            video_feature_dict[video_file] = {'features': features, 'label': label}
            print("Ready baby")

        return video_feature_dict

    def save_features(self, features, filename):
        np.save(f'{filename}_video_features.npy', features)

class EarlyFusionLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(EarlyFusionLayer, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

if __name__ == "__main__":
    feature_extractor = VisionFeatureExtractor("Emotion")
    video_folder_path = "/content/datasets/MELD/dev_splits_complete"
    video_path_csv = "/content/datasets/MELD/dev_sent_emo.csv"
    df = pd.read_csv(video_path_csv)
    df['video_id'] = "dia" + df['Dialogue_ID'].astype(str) + '_utt' + df['Utterance_ID'].astype(str) + '.mp4'
    print(df.head(3))

    extracted_features = feature_extractor.extract_features_from_folder(video_folder_path, df)
    feature_extractor.save_features(extracted_features, "train")



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 252MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed i

   Sr No.                                          Utterance Speaker  \
0       1     Oh my God, hes lost it. Hes totally lost it.  Phoebe   
1       2                                              What?  Monica   
2       3  Or! Or, we could go to the bank, close our acc...    Ross   

    Emotion Sentiment  Dialogue_ID  Utterance_ID  Season  Episode  \
0   sadness  negative            0             0       4        7   
1  surprise  negative            0             1       4        7   
2   neutral   neutral            1             0       4        4   

      StartTime       EndTime       video_id  
0  00:20:57,256  00:21:00,049  dia0_utt0.mp4  
1  00:21:01,927  00:21:03,261  dia0_utt1.mp4  
2  00:12:24,660  00:12:30,915  dia1_utt0.mp4  
Entering into the extractor
======['dia77_utt15.mp4', 'dia10_utt0.mp4', 'dia99_utt3.mp4', 'dia25_utt4.mp4', 'dia1_utt1.mp4', 'dia52_utt0.mp4', 'dia49_utt1.mp4', 'dia25_utt3.mp4', 'dia85_utt9.mp4', 'dia57_utt6.mp4', 'dia26_utt5.mp4', 'dia48_utt3.m

  0%|          | 1/1108 [00:11<3:24:42, 11.10s/it]

Ready baby


  0%|          | 2/1108 [00:11<1:31:38,  4.97s/it]

...

Ready baby


100%|█████████▉| 1107/1108 [41:50<00:02,  2.46s/it]

Ready baby


100%|██████████| 1108/1108 [41:53<00:00,  2.27s/it]

Ready baby


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import xgboost as xgb


def load_features(filename):
    data = np.load('/content/datasets/train_video_features.npy', allow_pickle=True).item()

    # Determine the maximum shape along each dimension
    max_shape = np.max([item['features'].shape for item in data.values()], axis=0)

    # Pad each feature array to have the max_shape, then flatten it
    padded_features = [
        np.pad(item['features'], ((0, max_shape[0] - item['features'].shape[0]),
                                  (0, max_shape[1] - item['features'].shape[1])))
        .flatten() for item in data.values()
    ]

    # Now, padded_features should be a list of 1D arrays of the same length
    # Stack them into a 2D array
    X = np.vstack(padded_features)
    y = np.array([item['label'] for item in data.values()])

    return X, y

from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix

from sklearn.multiclass import OneVsRestClassifier

def evaluate_classifiers(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Wrap classifiers with OneVsRestClassifier for multi-label classification
    classifiers = {
        'Random Forest': OneVsRestClassifier(RandomForestClassifier()),
        'SVM': OneVsRestClassifier(SVC()),
        'Logistic Regression': OneVsRestClassifier(LogisticRegression(max_iter=1000)),
        'XGBoost': OneVsRestClassifier(xgb.XGBClassifier())
    }

    for name, clf in classifiers.items():
        clf.fit(X_train, y_train)
        preds = clf.predict(X_test)
        print(f'{name} - Accuracy: {accuracy_score(y_test, preds)*100:.2f}%, '
              f'F1 Score: {f1_score(y_test, preds, average="weighted")*100:.2f}%')
if __name__ == "__main__":
    filename = 'train'  # Adjust the filename to match the output from your feature extraction script
    X, y = load_features(filename)
    evaluate_classifiers(X, y)


Random Forest - Accuracy: 7.66%, F1 Score: 12.92%


In [2]:
data = np.load('/content/datasets/train_video_features.npy', allow_pickle=True).item()

# Multimodal System

1. Early Fusion for Intra-Modality in Vision Features:
For each video, you've concatenated features from multiple architectures (ResNet, VGG16, ViT). Now, you can feed these concatenated features into a BLSTM layer to capture the temporal relations between frames.

In [ ]:
import torch
import torch.nn as nn

class EarlyFusionBLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(EarlyFusionBLSTM, self).__init__()
        self.blstm = nn.LSTM(input_dim, hidden_dim, bidirectional=True)

    def forward(self, x):
        out, _ = self.blstm(x)
        return out




2. Model-Specific Layers:
After the BLSTM, you can add one or more fully connected layers specific to the vision modality. Do the same for audio and text features.

In [ ]:
class ModalitySpecificLayers(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ModalitySpecificLayers, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

3.You can concatenate or use a weighted sum of the outputs from the modality-specific layers. Another option is to use attention mechanisms to dynamically weigh the importance of each modality.

In [ ]:
class LateFusionLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LateFusionLayer, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

4. Final Layers for Prediction:
Finally, you can have one or more layers to output the emotion classes.

In [ ]:
class EmotionClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(EmotionClassifier, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.fc(x)

Combining All:
You can then combine these components into a single model that takes the vision, audio, and text features as inputs and outputs the emotion class probabilities.

In [ ]:
class MultimodalEmotionClassifier(nn.Module):
    def __init__(self):
        super(MultimodalEmotionClassifier, self).__init__()
        self.early_fusion_blstm = EarlyFusionBLSTM(input_dim=26368, hidden_dim=512)
        self.vision_specific_layer = ModalitySpecificLayers(input_dim=1024, output_dim=256)  # 512 * 2 from BLSTM
        self.audio_specific_layer = ModalitySpecificLayers(input_dim=audio_dim, output_dim=256)
        self.text_specific_layer = ModalitySpecificLayers(input_dim=text_dim, output_dim=256)
        self.late_fusion_layer = LateFusionLayer(input_dim=768, output_dim=256)  # 256 * 3 from specific layers
        self.emotion_classifier = EmotionClassifier(input_dim=256, num_classes=num_emotion_classes)

    def forward(self, vision_input, audio_input, text_input):
        vision_features = self.early_fusion_blstm(vision_input)
        vision_features = self.vision_specific_layer(vision_features)

        audio_features = self.audio_specific_layer(audio_input)
        text_features = self.text_specific_layer(text_input)

        combined_features = torch.cat([vision_features, audio_features, text_features], dim=-1)
        combined_features = self.late_fusion_layer(combined_features)

        emotion_output = self.emotion_classifier(combined_features)
        return emotion_output


# Multimodal classifier

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class MultimodalClassifier(nn.Module):
    def __init__(self, audio_dim, text_dim, num_classes):
        super(MultimodalClassifier, self).__init__()

        self.audio_layer = nn.Linear(audio_dim, 128)
        self.text_layer = nn.Linear(text_dim, 128)

        self.fc_layer = nn.Linear(256, 128)  # 128 from audio + 128 from text
        self.classifier = nn.Linear(128, num_classes)

    def forward(self, audio_features, text_features):
        audio_out = self.audio_layer(audio_features)
        text_out = self.text_layer(text_features)

        combined = torch.cat((audio_out, text_out), dim=1)

        out = self.fc_layer(combined)
        out = self.classifier(out)

        return out

# Dimensions based on your feature extractors
audio_dim = 768  # for example
text_dim = 512  # for example
num_classes = 7  # Number of emotion classes

model = MultimodalClassifier(audio_dim, text_dim, num_classes)

# Sample data
audio_features = torch.randn(32, audio_dim)  # Batch of 32 samples
text_features = torch.randn(32, text_dim)    # Batch of 32 samples
labels = torch.randint(0, num_classes, (32,))  # Random labels

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Forward pass
outputs = model(audio_features, text_features)

# Compute loss and update weights
loss = criterion(outputs, labels)
loss.backward()
optimizer.step()


In [ ]:
import numpy as np

# Set the path to the .npy file
npy_file = "/content/datasets/MELD/dev_splits_complete/dia104_utt11.mp4_features.npy"

# Load the .npy file
data = np.load(npy_file)

# Print the data
print(data.shape)

(27, 26600)
